In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import math
from PIL import Image

##### Using super classes:

The following two cells respectively used the two methods in the email.

The first method still has more than 500 classes, so I didn't use it.  
However, for the second method:  
Although it has only 13 super-classes, they only include about 300 original classes, and the rest are not included. I have to put '\*' to the predictions which are not included in the 13 super-classes

In [ ]:
# first method
super_class = pd.read_csv('imagenet_hierarchy_of_categories.csv', index_col=None)
super_class.head()

In [ ]:
"""
# second method

# create a dataframe 'super_class' to map the subclasses to their superclasses
from robustness.tools.imagenet_helpers import common_superclass_wnid
from robustness.tools.imagenet_helpers import ImageNetHierarchy

in_path = 'imagenet'
in_info_path = 'imagenet_info'
in_hier = ImageNetHierarchy(in_path, in_info_path)


# option 1: specify number of super classes
# n_classes = 9
# superclass_wnid, class_ranges, label_map = in_hier.get_superclasses(n_classes,
#                                            # ancestor_wnid='n00001740',
#                                            # superclass_lowest=['n02084071'],
#                                            balanced=False)

# option 2: load pre-specified super classes (maximum nb of super classes is 16)
superclass_wnid = common_superclass_wnid('big_12')
class_ranges, label_map = in_hier.get_subclasses(superclass_wnid,
                                                 balanced=False)

# add fish to the super classes
fish_range = {0, 1, 2, 3, 4, 5, 6, 389, 390, 391, 392, 393, 394, 395, 396, 397}
class_ranges.append(fish_range)
label_map[12] = 'fish'

super_class = pd.DataFrame(columns = ['class', 'sup_class'])

for i, sub_classes in enumerate(class_ranges):
    for sub_class in sub_classes:
        new = pd.DataFrame({'class': sub_class, 'sup_class': label_map[i].split(',')[0]}, index = [1])
        super_class = super_class.append(new, ignore_index = True)
        
super_class = super_class.set_index('class')
super_class = super_class.sort_index()
# super_class.to_csv('super_class.csv')
"""
super_class = pd.read_csv('super_class.csv', index_col = 'class')
super_class

In [ ]:
def find_pair(class_range_):   
    pairs = np.empty(shape= [0, 2], dtype = int)
    for i in list(sorted(class_range))[0:-1]:
        pairs = np.concatenate((pairs, list(map(lambda x: (i, x), list(filter(lambda x: x>i, class_range_))))), axis=0)
    return pairs

In [ ]:
"""
dis_mat = np.ones((1000, 1000)) - np.identity(1000)

for n_class in range(1, 101):
    superclass_wnid, class_ranges, label_map = in_hier.get_superclasses(n_class,
                                               # ancestor_wnid='n00001740',
                                               # superclass_lowest=['n02084071'],
                                               balanced=False)
    pairs = np.empty(shape= [0, 2], dtype = int)
    for class_range in class_ranges:
        pairs = np.concatenate((pairs, find_pair(class_range)), axis=0)
    for pair in pairs:
        dis_mat[tuple(pair)] = 1/n_class
        
#     if (n_class%10 == 0) or (n_class == 100):
#         print(str(n_class) + '/100')
#         print(dis_mat)
        
# construct symmetric matrix
up = np.triu(dis_mat)
dis_mat = up + up.T

with open("imagenet_class_index.json") as f:
    imagenet_classes = {int(i):x[1] for i,x in json.load(f).items()}
    
dis_mat_save = pd.DataFrame(dis_mat, columns = list(imagenet_classes.values()), 
                            index = list(imagenet_classes.values()))
dis_mat_save.to_csv('dis_mat.csv')

dis_mat
"""

In [ ]:
# import pretrained models
alexnet = torchvision.models.alexnet(pretrained = True)
resnet = torchvision.models.resnet50(pretrained = True)
vgg = torchvision.models.vgg16(pretrained = True)

In [ ]:
# resnet_9 = torch.load('in9l_resnet50.pt') 
# resnet_9.eval()
# resnet_9l = torchvision.models.resnet50(pretrained = False)
aaa = torch.load('in9l_resnet50.pt') 
aaa

#############################################################################

In [ ]:
# import test data

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform_ = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

folder_dir = 'imagenet/only_fg/val_1'
data_test =  datasets.ImageFolder(root = folder_dir, transform = transform_,
                                       target_transform = None)

# show the test images

fig = plt.figure(figsize = (12, math.ceil(len(data_test)/20)))
fig.tight_layout()
for i in range(len(data_test)):
    img = data_test[i][0].permute(1, 2, 0)
    img_norm = (img - img.min()) / (img.max() - img.min())
    plt.subplot(math.ceil(len(data_test)/20), 20, i+1)
    plt.imshow(img_norm)
    plt.title(i)
    plt.axis('off')
plt.show()

In [ ]:
def cos_sim(vector_a, vector_b):
    
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

In [ ]:
# make prediction

pred_alexnet = []
pred_resnet = []
pred_vgg = []

diff_alexnet = []
diff_resnet = []
diff_vgg = []

logits_alexnet = None
logits_resnet = None
logits_vgg = None

alexnet.eval()
resnet.eval()
vgg.eval()

with open("imagenet_class_index.json") as f:
    imagenet_classes = {int(i):x[1] for i,x in json.load(f).items()}
    
# use super_class 
for i, data in enumerate(data_test):
    img = data[0].unsqueeze(0)
    
    logits_alexnet_last = logits_alexnet
    logits_alexnet = alexnet(img)
    pred = logits_alexnet.max(dim=1)[1].item()
    # if the predictions are not included in the superclasses, put '*' near them
    try:
        pred_class = super_class.loc[pred].sup_class
        pred_alexnet.append(pred_class)
    except:
        pred_alexnet.append(str(imagenet_classes[pred]) + '*')
    
    if (i%2) != 0:
        diff_alexnet.append(cos_sim(logits_alexnet.detach().numpy(),logits_alexnet_last.detach().numpy()))
    
    logits_resnet_last = logits_resnet
    logits_resnet = resnet(img)
    pred = logits_resnet.max(dim=1)[1].item()
    try:
        pred_class = super_class.loc[pred].sup_class
        pred_resnet.append(pred_class)
    except:
        pred_resnet.append(str(imagenet_classes[pred]) + '*')
    
    if (i%2) != 0:
        diff_resnet.append(cos_sim(logits_resnet.detach().numpy(),logits_resnet_last.detach().numpy()))
    
    logits_vgg_last = logits_vgg
    logits_vgg = vgg(img)
    pred = logits_vgg.max(dim=1)[1].item()
    try:
        pred_class = super_class.loc[pred].sup_class
        pred_vgg.append(pred_class)
    except:
        pred_vgg.append(str(imagenet_classes[pred]) + '*')
    if (i%2) != 0:
        diff_vgg.append(cos_sim(logits_vgg.detach().numpy(),logits_vgg_last.detach().numpy()))

In [ ]:
# diff_alexnet

In [ ]:
# record errors made

n_subclass = len(data_test.classes)
subclasses = data_test.classes[0:n_subclass]

err_alexnet = dict(map(lambda x: [x,[]], subclasses))
err_resnet = dict(map(lambda x: [x,[]], subclasses))
err_vgg = dict(map(lambda x: [x,[]], subclasses))

# use super_class 
for i in range(len(pred_alexnet)):
    if pred_alexnet[i] != data_test.classes[data_test[i][1]]:
    # if pred_alexnet[i] != imagenet_classes[data_test[i][1]]:
        err_alexnet[data_test.classes[data_test[i][1]]].append((i, pred_alexnet[i]))
    if pred_resnet[i] != data_test.classes[data_test[i][1]]:
    # if pred_resnet[i] != imagenet_classes[data_test[i][1]]:
        err_resnet[data_test.classes[data_test[i][1]]].append((i, pred_resnet[i]))
    if pred_vgg[i] != data_test.classes[data_test[i][1]]:
    # if pred_vgg[i] != imagenet_classes[data_test[i][1]]:
        err_vgg[data_test.classes[data_test[i][1]]].append((i, pred_vgg[i]))

print('errors made by alexnet: ' + '\n')
print(err_alexnet)
print('\n' + 'errors made by resnet: ' + '\n')
print(err_resnet)
print('\n' + 'errors made by vgg: ' + '\n')
print(err_vgg)

In [ ]:
# plot the errors made by alexnet and resnet, the titles represent the false predictions

for class_name in err_alexnet.keys():
    
    print('\n' + '\033[1m' + 'Errors made on {} by: '.format(class_name))
    fig = plt.figure(figsize = (14, math.ceil(len(err_alexnet[class_name])/3.5)))
    plt.suptitle('Alexnet', y = 1.01)
    plt.subplots_adjust(wspace = 0.1, hspace = 0.1)
    for i, data in enumerate(err_alexnet[class_name]):
        num, false_pred = data[0], data[1]
        img = data_test[num][0].permute(1, 2, 0)
        img_norm = (img - img.min()) / (img.max() - img.min())
        plt.subplot(math.ceil(len(err_alexnet[class_name])/10), 10, i+1)
        plt.imshow(img_norm)
        plt.title(str(num) + '\n' + false_pred, fontsize = 11)
        plt.axis('off')
    plt.show()    

    fig = plt.figure(figsize = (14, math.ceil(len(err_resnet[class_name])/3.5)))
    plt.suptitle('Resnet', y = 1.01)
    plt.subplots_adjust(wspace = 0.1, hspace = 0.1)
    for i, data in enumerate(err_resnet[class_name]):
        num, false_pred = data[0], data[1]
        img = data_test[num][0].permute(1, 2, 0)
        img_norm = (img - img.min()) / (img.max() - img.min())
        plt.subplot(math.ceil(len(err_resnet[class_name])/10), 10, i+1)
        plt.imshow(img_norm)
        plt.title(str(num) + '\n' + false_pred, fontsize = 11)
        plt.axis('off')
    plt.show()
    
    fig = plt.figure(figsize = (14, math.ceil(len(err_vgg[class_name])/3.5)))
    plt.suptitle('VGG', y = 1.01)
    plt.subplots_adjust(wspace = 0.1, hspace = 0.1)
    for i, data in enumerate(err_vgg[class_name]):
        num, false_pred = data[0], data[1]
        img = data_test[num][0].permute(1, 2, 0)
        img_norm = (img - img.min()) / (img.max() - img.min())
        plt.subplot(math.ceil(len(err_vgg[class_name])/10), 10, i+1)
        plt.imshow(img_norm)
        plt.title(str(num) + '\n' + false_pred, fontsize = 11)
        plt.axis('off')
    plt.show()

#### Quantify the influence of background

In [ ]:
logits_resnet_nb = logits_resnet[0:len(logits_resnet)/2]
logits_resnet_b = logits_resnet[len(logits_resnet):]



##### The remaining part is not related to the current task

Testing the performance on stylized images

In [ ]:
sti_name = 'filled-silhouettes'
# class_name = 'bear'

data_dir = 'texture-vs-shape-master/stimuli/' # + sti_name  + '/' + class_name + '/'

data_sti =  datasets.ImageFolder(root = data_dir, transform = transform_,
                                       target_transform = None)

In [ ]:
resnet.eval()
# pred_alex = []
plt.figure(figsize = (15, math.ceil(len(data_sti)/5)))

for i, data in enumerate(data_sti):
    img = data[0]
    pred = resnet(img.unsqueeze(0))
    pred_class = str(imagenet_classes[pred.max(dim=1)[1].item()])
    img_norm = (img - img.min()) / (img.max() - img.min())
    plt.subplot(math.ceil(len(data_sti)/10), 10, i+1)
    plt.imshow(img_norm.permute(1, 2, 0))
    plt.title(pred_class, fontsize = 11)
    plt.axis('off')
    if i == 600:
        break
plt.show()

In [ ]:
# deleted codes

"""
preprocess = transforms.Compose([
   transforms.Resi
   \ze((224, 224)),
   transforms.ToTensor(),
])

class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None,:,None,None]) / self.std.type_as(x)[None,:,None,None]

norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

data_dir = 'ILSVRC2012_img_val/ILSVRC2012_val_00000008.JPEG'
test_img = Image.open(data_dir)
test_img = preprocess(test_img)[None,:,:,:]
test_img_norm = norm(test_img)

plt.imshow(test_img[0].numpy().transpose(1,2,0))

alexnet.eval()

pred = alexnet(test_img_norm)
_, predicted = torch.max(pred, 1)

with open("imagenet_class_index.json") as f:
    imagenet_classes = {int(i):x[1] for i,x in json.load(f).items()}
print('predicted label:{} '.format(int(predicted)) + str(imagenet_classes[pred.max(dim=1)[1].item()]))
print('true label:{} '.format(int(labels[0][7])) + str(imagenet_classes[int(labels[0][7])]))
"""